<a href="https://colab.research.google.com/github/isegura/OCW-UC3M-NLPDeep-2023/blob/main/tema5_7_pipelines.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<center>
<img src="https://upload.wikimedia.org/wikipedia/commons/4/47/Acronimo_y_nombre_uc3m.png" width=50%/>

<h1><font color='#12007a'>Procesamiento de Lenguaje Natural con Aprendizaje Profundo</font></h1>
<p>Autora: Isabel Segura Bedmar</p>

<img align='right' src="https://mirrors.creativecommons.org/presskit/buttons/88x31/png/by-nc-sa.png" width=15%/>
</center>   

# 5.7. Cómo usar un transformer para inferir (pipelines)

Además de las fases de pre-training y fine-tuning, también podríamos hablar de la fase de inferencia, donde el modelo es utilizado para producir la salida sobre un texto nuevo para una tarea concreta.


## Pipelines

La librería **transformer** proporciona una clase **Pipeline** que permite cargar un modelo pre-entrenado para una tareas de PLN específica como: clasificación de textos, análisis de sentimiento, reconocimiento de entidades, etc.



In [1]:
!pip install -q transformers


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 47.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 28.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 89.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 58.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 19.9 MB/s eta 0:00:00


## Análisis de sentimiento
El siguiente pipeline te permite leer un texto y le va a asignar una polaridad:

In [9]:
from transformers import pipeline
pipe = pipeline('sentiment-analysis')

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


In [11]:
print(pipe("The movie was very interesting."))
print(pipe("This is the worst movie ever made."))


[{'label': 'POSITIVE', 'score': 0.9998030066490173}]
[{'label': 'NEGATIVE', 'score': 0.9997480511665344}]


Vemos que los resultados son bastante buenos. El problema es que no tiene una clase neutra, porque fue entrenado sobre un dataset que únicamente tenía dos clases (SST-2).

Vamos a cargar el mismo pipeline pero utilizando un modelo para español. En concreto, usaremos un modelo BETO (BERT entrenado con textos en español) ajustado para esta tarea:

In [12]:
pipe = pipeline('sentiment-analysis', model="finiteautomata/beto-sentiment-analysis")
pipe(['La película era muy interesante',
               'Las clases serán los lunes', 'Esta es la peor película jamas hecha',])

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


[{'label': 'POS', 'score': 0.9985098242759705},
 {'label': 'NEU', 'score': 0.998573899269104},
 {'label': 'NEG', 'score': 0.9990886449813843}]

Los resultados siguen siendo bastante buenos. En este caso, sí tenemos clase neutra.

## Zero-shot-classification

Este pipeline puede ser aplicado a distintas tareas de clasificación. Para ello es únicamente necesario pasarle el conjunto de clases.

In [13]:
zero_pipeline = pipeline("zero-shot-classification")


No model was supplied, defaulted to facebook/bart-large-mnli and revision c626438 (https://huggingface.co/facebook/bart-large-mnli).
Using a pipeline without specifying a model name and revision in production is not recommended.


### Clasificación de noticias
Devuelve las clases ordenadas por su probabilidad. En el siguiente ejemplo, la clase más probable es que el texto sea una noticia sobre salud:

In [14]:
zero_pipeline(
    "Las vacunas para el COVID ayudaron a controlar la enfermedad.",
    candidate_labels=["educación", "política", "economía", "salud"],
)

{'sequence': 'Las vacunas para el COVID ayudaron a controlar la enfermedad.',
 'labels': ['salud', 'política', 'educación', 'economía'],
 'scores': [0.7637993097305298,
  0.16598063707351685,
  0.0376877561211586,
  0.03253228962421417]}

### Detección de mensajes de odio

In [15]:
zero_pipeline(
    "Todos los musulmanes son terroristas.",
    candidate_labels=["racista", "sexista", "neutro"],
)

{'sequence': 'Todos los musulmanes son terroristas.',
 'labels': ['racista', 'neutro', 'sexista'],
 'scores': [0.9711172580718994, 0.01899351179599762, 0.009889206849038601]}

In [16]:
zero_pipeline(
    "Mujeres conducen peor que los hombres.",
    candidate_labels=["racista", "sexista", "neutro"],
)

{'sequence': 'Mujeres conducen peor que los hombres.',
 'labels': ['sexista', 'racista', 'neutro'],
 'scores': [0.8253806233406067, 0.12508924305438995, 0.04953011870384216]}

### Identificar ideología de un tweet:

In [17]:
zero_pipeline(
    "El aborto es un asesinato.",
    candidate_labels=["derecha", "izquierda", "neutro"],
)

{'sequence': 'El aborto es un asesinato.',
 'labels': ['derecha', 'izquierda', 'neutro'],
 'scores': [0.7093948125839233, 0.188075453042984, 0.10252974927425385]}

### Detectar fake news

In [18]:
zero_pipeline(
    "The Earth is flat",
    candidate_labels=["fake", "non-fake"],
)

{'sequence': 'The Earth is flat',
 'labels': ['fake', 'non-fake'],
 'scores': [0.713068425655365, 0.2869315445423126]}

El pipeline parece clasificar con bastante acierto, aunque sean tareas de clasificación distintas. Aunque no siempre funciona correctamente:


In [19]:
zero_pipeline(
    "Donald Trump is a left-wing politician",
    candidate_labels=["fake", "non-fake"],
)

{'sequence': 'Donald Trump is a left-wing politician',
 'labels': ['non-fake', 'fake'],
 'scores': [0.6424107551574707, 0.3575892150402069]}

Ejemplo de noticia false (https://edition.cnn.com/2022/03/12/politics/fact-check-dicaprio-donation-10-million-ukraine/index.html)

In [20]:
zero_pipeline(
    "Leonardo DiCaprio makes multiple donations to Ukraine",
    candidate_labels=["fake", "non-fake"],
)

{'sequence': 'Leonardo DiCaprio makes multiple donations to Ukraine',
 'labels': ['non-fake', 'fake'],
 'scores': [0.9621754884719849, 0.03782454505562782]}

## Generación de textos

El siguiente pipeline te permitirá continuar un texto:


In [21]:
generator = pipeline("text-generation", model='DeepESP/gpt2-spanish-medium', pad_token_id = 50256)


Es posible indicar el número de salidas generadas y su longitud máxima:

In [22]:
generator("Erase una vez una niña", max_length=15, num_return_sequences=3)

[{'generated_text': 'Erase una vez una niña, y ahora también una mujer. \n\n'},
 {'generated_text': 'Erase una vez una niña. \n\nLos niños le dieron unas palmaditas'},
 {'generated_text': 'Erase una vez una niña… pero se llama Eva…, no…, es'}]

## Mask-filling




Otra tarea relacionada con la generación de textos, es la tarea capaz de inferir los huecos en una oración. El pipeline es **fill-mask**.

In [23]:
unmasker = pipeline("fill-mask")

No model was supplied, defaulted to distilroberta-base and revision ec58a5b (https://huggingface.co/distilroberta-base).
Using a pipeline without specifying a model name and revision in production is not recommended.


Some weights of the model checkpoint at distilroberta-base were not used when initializing RobertaForMaskedLM: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


El hueco a inferir debe marcarse con  el token especial  $<mask>$.

**top_k** indica el número de respuestas que queremos obtener:

In [24]:
unmasker("We will study how to develop <mask> models.", top_k=3)

[{'score': 0.1785462647676468,
  'token': 27930,
  'token_str': ' predictive',
  'sequence': 'We will study how to develop predictive models.'},
 {'score': 0.07322675734758377,
  'token': 30412,
  'token_str': ' mathematical',
  'sequence': 'We will study how to develop mathematical models.'},
 {'score': 0.04494432732462883,
  'token': 38163,
  'token_str': ' computational',
  'sequence': 'We will study how to develop computational models.'}]

Cómo funciona si hay varios huecos:

In [25]:
unmasker("In this <mask>, we will study how to <mask> models.", top_k=2)

[[{'score': 0.3208775818347931,
   'token': 1566,
   'token_str': ' article',
   'sequence': '<s>In this article, we will study how to<mask> models.</s>'},
  {'score': 0.2518361508846283,
   'token': 2225,
   'token_str': ' paper',
   'sequence': '<s>In this paper, we will study how to<mask> models.</s>'}],
 [{'score': 0.07576775550842285,
   'token': 1119,
   'token_str': ' build',
   'sequence': '<s>In this<mask>, we will study how to build models.</s>'},
  {'score': 0.0601893775165081,
   'token': 22016,
   'token_str': ' optimize',
   'sequence': '<s>In this<mask>, we will study how to optimize models.</s>'}]]

## Reconocimiento de entidades
También  existen pipelines disponibles para la tarea de reconocimiento de entidades, que puedes usar directamente sin necesidad de entrenar un modelo para la tarea:

In [29]:
ner = pipeline("ner", model="mrm8488/bert-spanish-cased-finetuned-ner",
    tokenizer=('mrm8488/bert-spanish-cased-finetuned-ner',  {"use_fast": False}
))


Some weights of the model checkpoint at mrm8488/bert-spanish-cased-finetuned-ner were not used when initializing BertForTokenClassification: ['bert.pooler.dense.weight', 'bert.pooler.dense.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Com el parámetreo grouped_entities a True, te permite identificar a nivel de sintagma (si la entidad es multitoken)

In [30]:
ner('La FIFA ha elegido a Arabia Saudí como sede del Mundial de Clubes de 2023', grouped_entities = True)

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/token_classification.py:169: UserWarning: `grouped_entities` is deprecated and will be removed in version v5.0.0, defaulted to `aggregation_strategy="simple"` instead.
  warnings.warn(
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


[{'entity_group': 'ORG',
  'score': 0.9986659,
  'word': 'FIFA',
  'start': None,
  'end': None},
 {'entity_group': 'LOC',
  'score': 0.99912256,
  'word': 'Arabia Saudí',
  'start': None,
  'end': None},
 {'entity_group': 'MISC',
  'score': 0.9992069,
  'word': 'Mundial de Clubes',
  'start': None,
  'end': None}]

Si no se utiliza dicho argumento, entonces la anotación será en formatio BIO (B-type para el principio de una entidad, I-type para un token interno, y O para un token que no pertenece a una entidad):

In [31]:
ner('La FIFA ha elegido a Arabia Saudí como sede del Mundial de Clubes de 2023')

[{'entity': 'B-ORG',
  'score': 0.9986659,
  'index': 2,
  'word': 'FIFA',
  'start': None,
  'end': None},
 {'entity': 'B-LOC',
  'score': 0.9991411,
  'index': 6,
  'word': 'Arabia',
  'start': None,
  'end': None},
 {'entity': 'I-LOC',
  'score': 0.9991079,
  'index': 7,
  'word': 'Sa',
  'start': None,
  'end': None},
 {'entity': 'I-LOC',
  'score': 0.9991186,
  'index': 8,
  'word': '##udí',
  'start': None,
  'end': None},
 {'entity': 'B-MISC',
  'score': 0.99966,
  'index': 12,
  'word': 'Mundial',
  'start': None,
  'end': None},
 {'entity': 'I-MISC',
  'score': 0.999062,
  'index': 13,
  'word': 'de',
  'start': None,
  'end': None},
 {'entity': 'I-MISC',
  'score': 0.9992906,
  'index': 14,
  'word': 'Club',
  'start': None,
  'end': None},
 {'entity': 'I-MISC',
  'score': 0.9988152,
  'index': 15,
  'word': '##es',
  'start': None,
  'end': None}]

Si quieres saber más sobre pipelines, por favor, consulta la siguiente página https://huggingface.co/docs/transformers/main_classes/pipelines
